In [99]:
#EXTRACCION TRANSFORMACION Y CARGA DE DATOS (ETL)

# Importar las librerías para el análisis de los datos
from github import Github
from pprint import pprint
import requests
import pandas as pd
import time
from urllib.request import urlopen
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import LabelEncoder


# Leer los datos directamente desde GitHub

#Archivos json
url="https://raw.githubusercontent.com/mayteet/PI01_DATA05/main/Datasets/netflix_titles.json"
f=requests.get(url)
data=f.json()

# Convertir el archivo de json en un DataFrame
netflix=pd.DataFrame(data)

#Archivos csv
amazon= pd.read_csv("https://raw.githubusercontent.com/mayteet/PI01_DATA05/main/Datasets/amazon_prime_titles.csv")
disney= pd.read_csv("https://raw.githubusercontent.com/mayteet/PI01_DATA05/main/Datasets/disney_plus_titles.csv")
hulu= pd.read_csv("https://raw.githubusercontent.com/mayteet/PI01_DATA05/main/Datasets/hulu_titles.csv") 

#Agregamos la columna de plataform a cada una de las tablas y las unimos
netflix["plataform"]= "netflix"
amazon["plataform"]="amazon"
disney["plataform"]="disney"
hulu["plataform"]="hulu"

#Unimos los dataframes
df=pd.concat([netflix, amazon, disney, hulu]) 

#Análisis exploratorio de los datos


#Arreglo de la base de datos de la columna de duración
#Creamos un dataframe donde guardaremos los datos de duración que estaban ubicados en la columna rating
data_all=pd.DataFrame([])

#Identificamos la lista de indicadores de data que es de duración
Lista_wrong=["min", "Seasons","Season"]

#Filtramos el dataframe principal con las columnas que queremos arreglar
df_analysis= df[["title","rating","plataform"]]

#Realizamos un ciclo iterativo para identificar los datos de la columna rating que deberían ir en duración
for elemento in Lista_wrong:
    
    # Filtrar el Dataframe por columna rating en el que se encuentre la palabra min, Seasons, Season
    df_analysis_filter= df_analysis.loc[df_analysis["rating"].str.contains(elemento, na=False)]
    data_all=pd.concat([data_all,df_analysis_filter])

#Renombramos la columna
data_all.rename(columns={'rating':"duration"}, inplace=True)


#Identificamos de la columna de duración del dataframe principal cuáles eran sus valores no nulos
#Sacamos las variables del dataframe principal
df_analysis_2= df[["title","duration","plataform"]]
df_analysis_2=df_analysis_2.dropna(subset="duration")

#Juntamos todos los datos de duración en una sóla columna
data_all=pd.concat([data_all, df_analysis_2])

#Guardamos todos esos valores en otra columna para no sobreponer y/o perder información
data_all.rename(columns={'duration':"duration_2"}, inplace=True)

#Quitamos las cadenas de texto para tener valores numéricos en duración y poder analizar correctamente los valores máximos y mínimos
data_all["duration_2"]=data_all["duration_2"].str.replace(" min","")
data_all["duration_2"]=data_all["duration_2"].str.replace(" Seasons","")
data_all["duration_2"]=data_all["duration_2"].str.replace(" Season","")

#Juntamos todo en la tabla principal
df_final= pd.merge(df,data_all, on=["title","plataform"],how="left")

#Quitamos los duplicados de la tabla
df_final=df_final.drop_duplicates(["title","type","plataform"])



#Construimos la tabla types
df_types=pd.DataFrame([])
df_types["type"]=pd.DataFrame(df_final["type"].unique())
df_types["type_id"]=1,2
df_types= df_types.set_index("type")
df_types.to_csv("types")


#Construimos la tabla shows
df_shows= df_final.drop_duplicates(subset=["title","type"])
df_shows=df_shows[["title","type","director","cast","country","release_year"]]
df_shows["title_id"]= df_shows.reset_index().index
df_shows["cast"]=df_shows["cast"].str.replace(",",";")
df_shows= pd.merge(df_shows,df_types, on="type",how="left")
df_shows=df_shows.set_index(["title","type"])
df_shows.to_csv("shows")

#Construimos la tabla plataform
df_platform=pd.DataFrame([])
df_platform["plataform"]=pd.DataFrame(df_final["plataform"].unique())
df_platform["platform_id"]= df_platform.reset_index().index
df_platform=df_platform.set_index("plataform")
df_platform.to_csv("plataform")

#Construimos la tabla shows_in_platform
#Filtramos la informacioón que debe contener
df_shows_in_platform=df_final[["title","type","plataform","show_id","duration_2","rating","date_added","listed_in"]]

#Codificamos con las tablas previamente hechas
df_shows_in_platform=pd.merge(df_shows_in_platform,df_platform, on="plataform",how="left")
df_shows_in_platform=pd.merge(df_shows_in_platform,df_types, on="type",how="left")
df_shows_in_platform=df_shows_in_platform.set_index(["title","type"])
df_shows_in_platform=pd.merge(df_shows_in_platform,df_shows[["title_id"]], on=["title","type"],how="left")

#Eliminamos las columnas que no nos sirvan para poder tener nuestro modelo de datos relacional, y guardados en archivos csv.




In [107]:
df_consul_1= df_shows_in_platform[["plataform","duration_2"]]
df_consul_2= df_shows[["release_year"]]

df_consul_ej1=pd.merge(df_consul_1,df_consul_2, on=["title","type"],how="left")

df_consul_ej1.reset_index(level=1, inplace=True, col_level=0)

df_consul_ej1=df_consul_ej1[df_consul_ej1["type"]=="Movie"]
df_consul_ej1["duration_2"]=df_consul_ej1["duration_2"].astype("Int64")
df_consul_ej1

,type,plataform,duration_2,release_year
title,,,,
Dick Johnson Is Dead,Movie,netflix,90,2020
My Little Pony: A New Generation,Movie,netflix,91,2021
Sankofa,Movie,netflix,125,1993
The Starling,Movie,netflix,104,2021
Je Suis Karl,Movie,netflix,127,2021
...,...,...,...,...
Getting On,Movie,hulu,3,2009
(Sub) Ninja Scroll,Movie,hulu,93,1993
Packed to the Rafters,Movie,hulu,6,2008


In [108]:
df_consul_ej1

,type,plataform,duration_2,release_year
title,,,,
Dick Johnson Is Dead,Movie,netflix,90,2020
My Little Pony: A New Generation,Movie,netflix,91,2021
Sankofa,Movie,netflix,125,1993
The Starling,Movie,netflix,104,2021
Je Suis Karl,Movie,netflix,127,2021
...,...,...,...,...
Getting On,Movie,hulu,3,2009
(Sub) Ninja Scroll,Movie,hulu,93,1993
Packed to the Rafters,Movie,hulu,6,2008


In [109]:
df_consul_ej1[df_consul_ej1.groupby(["release_year", "plataform"])["duration_2"].mean()]


KeyError: "None of [Index([             72.0,             135.0,             133.0,\n                   113.0,             118.0,            81.375,\n                    56.5,              65.0,               8.0,\n                   82.25,\n       ...\n       88.90776699029126, 93.46603475513429, 86.08163265306122,\n       66.05405405405405,  82.6067415730337, 92.14119922630562,\n       85.97431355181577, 45.81428571428572, 88.34545454545454,\n       96.44404332129965],\n      dtype='object', length=335)] are in the [columns]"

In [104]:
df_consul_ej1

release_year  plataform
1920          amazon            72.0
              disney           135.0
1922          amazon           133.0
1923          amazon           113.0
1924          amazon           118.0
                             ...    
2020          netflix      92.141199
2021          amazon       85.974314
              disney       45.814286
              hulu         88.345455
              netflix      96.444043
Name: duration_2, Length: 335, dtype: Float64